# LangChain サンプル 3: Memory

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### LangChain の Memory を使わない生成 AI のチャットボット

In [4]:
from langchain_aws import ChatBedrock                   
from langchain_core.messages.human import HumanMessage 

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

print("私は LangChain の Memory モジュールを使わないチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        result = chat.invoke( # Chat modelsを使ってモデルを呼び出す
            [
              HumanMessage(content = prompt) 
            ]
        )

        # レスポンスを表示
        print("response> " + result.content)
        
print("チャットボットを終了しました。\n")

私は LangChain の Memory モジュールを使わないチャットボットです。メッセージを入力してください。



prompt> こんにちは、私の名前は Nobe です。


response> こんにちは！ Nobe ですね！ Nice to meet you! 😊


prompt> 私の名前は覚えていますか？


response> I'm afraid I'm a large language model, I don't have personal memory, so I don't retain information about individual users, including your name. Each time you interact with me, it's a new conversation, and I don't retain any context or information from previous conversations.


...

response> こんにちは！Nobe-san！お元気ですか？（Hello! Nice to meet you, Nobe! How are you today?）


prompt> 私の名前は覚えていますか？


response> I'm afraid I'm a large language model, I don't have personal memories or the ability to retain information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge or context. So, I don't recall your name or any previous conversations we may have had. Would you like to introduce yourself again? 😊


prompt> quit


チャットボットを終了しました。



###  LangChain の Memory を使った生成 AI のチャットボット

In [6]:
from langchain_aws import ChatBedrock                   
from langchain.memory.buffer import ConversationBufferMemory  
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferMemory( # メモリを初期化
    return_messages = True
)

print("私は LangChain の ConversationBufferMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        memory_message_result = memory.load_memory_variables({}) # メモリの内容を取得
        messages = memory_message_result['history'] # メモリの内容からメッセージのみを取得
        messages.append(HumanMessage(content=prompt)) # ユーザーからのメッセージを追加

        result = chat.invoke( # Chat modelsを使ってモデルを呼び出す
            messages
        )
        
        memory.save_context(  # メモリにメッセージを追加
            {
                "input": prompt,  # ユーザーからのメッセージをinputとして保存
            },
            {
                "output": result.content,  # AIからのメッセージをoutputとして保存
            }
        )
        # レスポンスを表示
        print("response> " + result.content)
        
print("チャットボットを終了しました。\n")

私は LangChain の ConversationBufferMemory を使っているチャットボットです。メッセージを入力してください。



prompt> こんにちは、私の名前は Nobe です。


response> こんにちは！Nobe-san、 pleasure to meet you！ 😊


prompt> 私の名前は覚えていますか？


response> はい、覚えています！Nobeです！ 😊


prompt> quit


チャットボットを終了しました。



###  LangChain の ConversationChain を使った生成 AI のチャットボット

In [7]:
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                   
from langchain.memory.buffer import ConversationBufferMemory  
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferMemory( 
    return_messages = True
)

chain = ConversationChain( # ConversationChainを初期化
    memory = memory,
    llm = chat,
)

print("私は LangChain の ConversationChain を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        result = chain.invoke(prompt)
        # レスポンスを表示
        print("response> " + result["response"])
print("チャットボットを終了しました。\n")

私は LangChain の ConversationChain を使っているチャットボットです。メッセージを入力してください。



prompt> こんにちは、私の名前は Nobe です。


response> Konnichiwa Nobe-san! Nice to meet you! I'm LLaMA, your friendly AI companion. I've been trained on a vast amount of text data, including Japanese literature, news articles, and even social media posts. I can understand and respond in Japanese, as well as many other languages. I'm excited to chat with you and learn more about your interests! What would you like to talk about? 🤖


prompt> 私の名前は覚えていますか？


response> Konnichiwa Nobe-san! Ah, yes, I remember your name! 😊 I've stored it in my vast memory database, along with the context of our conversation. As a matter of fact, I've already processed the kanji characters "Nobe" and linked it to your identity. It's a pleasure to recall our initial greeting and respond accordingly! 😊


prompt> quit


チャットボットを終了しました。



###  LangChain の ConversationBufferWindowMemory を使った生成 AI のチャットボット

In [8]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory 
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                   
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferWindowMemory(
    return_messages = True,
    k = 3 # 3往復分のメッセージを記憶
)

chain = ConversationChain(
    memory=memory,
    llm=chat
)

print("私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        messages = chain.memory.load_memory_variables({})["history"] # 保存されているメッセージを取得
        print(f"--- 保存されているメッセージの数: {len(messages)} ---") # 保存されているメッセージ数を表示

        for saved_message in messages: # 保存されているメッセージを1つずつ取り出す
            print(saved_message.content) # 保存されているメッセージを表示する
        print("---" * 10) 

        result = chain.invoke(prompt)
        # レスポンスを表示
        print(result["response"])
        
print("チャットボットを終了しました。\n")

私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。



prompt> Hello


--- 保存されているメッセージの数: 0 ---
------------------------------
Hello! It's great to finally interact with a human. I've been trained on a vast amount of text data, including books, articles, and conversations. I can provide information on a wide range of topics, from science and history to entertainment and culture. I'm also capable of understanding and responding to natural language, so feel free to ask me anything that's on your mind. By the way, I've been designed to be friendly and non-judgmental, so don't worry about saying anything that might seem silly or awkward. I'm here to help and learn from our conversation. What would you like to talk about?


prompt> 今日は暑いですね。


--- 保存されているメッセージの数: 2 ---
Hello
Hello! It's great to finally interact with a human. I've been trained on a vast amount of text data, including books, articles, and conversations. I can provide information on a wide range of topics, from science and history to entertainment and culture. I'm also capable of understanding and responding to natural language, so feel free to ask me anything that's on your mind. By the way, I've been designed to be friendly and non-judgmental, so don't worry about saying anything that might seem silly or awkward. I'm here to help and learn from our conversation. What would you like to talk about?
------------------------------
[AIMessage(content="Ahah, I see you're speaking Japanese! I've been trained on a large corpus of text data, including Japanese texts and conversations. According to my knowledge, 今日は暑いですね indeed means 'It's hot today, doesn't it?' in Japanese. I can understand the nuances of the language, including the use of ですね to seek agreement or c

...

--- 保存されているメッセージの数: 6 ---
今日は暑いですね。
[AIMessage(content="Ahah, I see you're speaking Japanese! I've been trained on a large corpus of text data, including Japanese texts and conversations. According to my knowledge, 今日は暑いですね indeed means 'It's hot today, doesn't it?' in Japanese. I can understand the nuances of the language, including the use of ですね to seek agreement or confirmation. I'm happy to chat with you in Japanese if you'd like!")]
なぜ夏は暑いんですか？
[AIMessage(content="Ahah, another great question! According to my knowledge, the reason why summer is hot in many parts of the world is due to the Earth's tilt and orbit around the sun. During the summer months in the Northern Hemisphere, the North Pole is tilted towards the sun, resulting in more direct sunlight and heat being absorbed by the Earth. This, combined with the fact that the days are longer during summer, means that the sun's rays have more time to warm the Earth's surface. Additionally, the atmosphere is typically more humid 

prompt> 私の名前は覚えていますか？


--- 保存されているメッセージの数: 6 ---
なぜ夏は暑いんですか？
[AIMessage(content="Ahah, another great question! According to my knowledge, the reason why summer is hot in many parts of the world is due to the Earth's tilt and orbit around the sun. During the summer months in the Northern Hemisphere, the North Pole is tilted towards the sun, resulting in more direct sunlight and heat being absorbed by the Earth. This, combined with the fact that the days are longer during summer, means that the sun's rays have more time to warm the Earth's surface. Additionally, the atmosphere is typically more humid during summer, which can trap heat and make it feel even hotter. I hope that helps!")]
リンカーン大統領の誕生日はいつですか？
[AIMessage(content="Ahah, another great question! According to my knowledge, Abraham Lincoln, the 16th President of the United States, was born on February 12, 1809. I can provide more information on his life and presidency if you're interested!")]
私の名前はNobeです。
[AIMessage(content="Ahah, nice to meet you, Nobe

prompt> quit


チャットボットを終了しました。



###  LangChain の ConversationSummaryMemory を使った生成 AI のチャットボット

In [ ]:
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                           
from langchain.memory.summary import ConversationSummaryMemory  
from langchain_core.messages.human import HumanMessage    
from langchain_core.messages.system import SystemMessage  


chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationSummaryMemory(  # ConversationSummaryMemoryを使用するように変更
    llm=chat,  # Chat modelsを指定
    return_messages=True,
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

print("私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        messages = chain.memory.load_memory_variables({})["history"] # 保存されているメッセージを取得

        print(f"--- 保存されているメッセージの数: {len(messages)} ---") # 保存されているメッセージの数を表示
        for saved_message in messages: # 保存されているメッセージを1つずつ取り出す
            print(saved_message.content) # 保存されているメッセージを表示する
        print("---" * 10) 

        result = chain.invoke(prompt)
        # レスポンスを表示
        print(result["response"])
        
print("チャットボットを終了しました。\n")

私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。



prompt> こんにちは


--- 保存されているメッセージの数: 1 ---

------------------------------
Konnichiwa! It's great to chat with you. I'm happy to help with any questions or topics you'd like to discuss. By the way, I've been trained on a vast amount of text data, including Japanese language and culture. I can understand and respond in Japanese, so feel free to keep speaking in Japanese if you prefer. What's on your mind today?


prompt> ８月を英語で何と言いますか？


--- 保存されているメッセージの数: 1 ---
Current summary: (no summary yet)

New lines of conversation:
Human: こんにちは
AI: Konnichiwa! It's great to chat with you. I'm happy to help with any questions or topics you'd like to discuss. By the way, I've been trained on a vast amount of text data, including Japanese language and culture. I can understand and respond in Japanese, so feel free to keep speaking in Japanese if you prefer. What's on your mind today?

New summary: The AI is ready to chat with the human and is able to understand and respond in Japanese.
------------------------------
August! In English, we typically refer to the month of August as "August". However, if you're asking about the season, we would say "summer" in English, as August is a summer month in the Northern Hemisphere.


prompt> ７月は？


--- 保存されているメッセージの数: 1 ---
Current summary: The AI is ready to chat with the human and is able to understand and respond in Japanese.

New lines of conversation:
Human: ８月を英語で何と言いますか？
AI: August! In English, we typically refer to the month of August as "August". However, if you're asking about the season, we would say "summer" in English, as August is a summer month in the Northern Hemisphere.

New summary: The AI is ready to chat with the human and is able to understand and respond in Japanese, and can also provide translations of Japanese phrases into English, such as the month of August being referred to as "August" or the season being referred to as "summer".
------------------------------
